In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import warnings
import contextlib
from datetime import datetime
import torch.multiprocessing as mp
import sys
mp.set_start_method('spawn')

In [2]:
from utils.datasets import *
from utils.eval import *
from utils.model import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_names = [
    "resnet50",
    "resnet18", "alexnet", "vgg16",
    "densenet", "inception", "googlenet", 
    "resnext50_32x4d", "wide_resnet50_2"
]

cuda


In [4]:
# freezed model
def get_original_model(net):
    if net == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        # models.resnet18(pretrained=True)
    elif net == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        # model = models.resnet18(pretrained=True)
    elif net == "alexnet":
        # model = models.alexnet(pretrained=True)
        model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
    elif net == "vgg16":
        model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        # model = models.vgg16(pretrained=True)
    elif net == "densenet":
        model = models.densenet161(weights=models.DenseNet161_Weights.DEFAULT)
        # model = models.densenet161(pretrained=True)
    elif net == "inception":
        model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
        # model = models.inception_v3(pretrained=True)
    elif net == "googlenet":
        model = models.googlenet(weights=models.GoogLeNet_Weights.DEFAULT)
        # model = models.googlenet(pretrained=True)
    elif net == "resnext50_32x4d":
        model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.DEFAULT)
        # model = models.resnext50_32x4d(pretrained=True)
    elif net == "wide_resnet50_2":
        model = models.wide_resnet50_2(weights=models.Wide_ResNet50_2_Weights.DEFAULT)
        # model = models.wide_resnet50_2(pretrained=True)
    else:
        raise ValueError("Invalid model name")
    for param in model.parameters():
        param.requires_grad = False
    # Remove the last fully connected layer
    if "inception" in net:
        model.aux_logits=False
        num_ftrs = model.fc.in_features
        model.fc = nn.Identity()
        model.fc = nn.Linear(2048, num_classes)
        model.fc.requires_grad = True
        optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
    if "alexnet" in net or "vgg" in net or  "mobilenet_v3_large" in net:
        last_fc_layer = model.classifier[-1]
        num_ftrs = last_fc_layer.in_features
        model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
        model.classifier[-1].requires_grad = True
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    elif "densenet" in net:
        last_fc_layer = model.classifier
        num_ftrs = last_fc_layer.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)
        model.classifier.requires_grad = True
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    else:
        num_ftrs = model.fc.in_features
        model.fc = nn.Identity()
        model.fc.requires_grad = True
        model.fc = nn.Linear(num_ftrs, num_classes)
        optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    return model,criterion, optimizer

In [5]:
num_epochs = 50
def train_net(net, test_ratio=0.2):
    model, criterion, optimizer = get_original_model(net)
    original_stderr = sys.stderr
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    tps, tns, fps, fns = [], [], [], []
    current_date = datetime.now().strftime('%Y-%m-%d')
    save_dir = os.path.join("saved_models/tl+rotation+more", current_date)
    os.makedirs(save_dir, exist_ok=True)
    
    sys.stderr = open(os.devnull, 'w')
    
    for timestamp in range(1, 290):
    # for timestamp in range(50, 53):
        debug = True
        if timestamp in  [1, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 289]:
            debug = True
        if debug:
            print("timestamp: ", timestamp)
        if "inception" in net:
            dataloaders = get_dataloaders(timestamp, test_ratio=test_ratio, inception=True, data_enrich=True)
        else:
            dataloaders = get_dataloaders(timestamp, test_ratio=test_ratio, data_enrich=True)
        model_trained, loss_values = train_model(model, criterion, optimizer, dataloaders, num_epochs=num_epochs, debug=True, inception=("inception" in net), output=debug)
        model_filename = os.path.join(save_dir, f"model_{net}_time_{timestamp}_epochs{num_epochs}_{test_ratio}.pt")
        torch.save(model_trained.state_dict(), model_filename)
    
        accuracy, precision, recall, f1, tp, tn, fp, fn = get_metrics(model_trained, dataloaders['test'], inception=("inception" in net))
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        if debug:
            print("Accuracy: ", accuracy)
        recall_list.append(recall)
        f1_list.append(f1)
        tps.append(tp)
        tns.append(tn)
        fps.append(fp)
        fns.append(fn)
    sys.stderr = original_stderr
    return accuracy_list, precision_list, recall_list, f1_list, tps, tns, fps, fns

In [6]:
# for model in 
# accuracy_list, precision_list, recall_list, f1_list, tps, tns, fps, fns = train_net("googlenet")
model_names = [
    # "resnet50",
    # "resnet18", "alexnet", 
    # "vgg16",
    "densenet", 
    # "inception", 
    # "googlenet", 
    # "resnext50_32x4d", 
    # "wide_resnet50_2"
]
acc_map = {}
pre_map = {}
rec_map = {}
f1_map = {}
tps_map = {}
tns_map = {}
fps_map = {}
fns_map = {}
test_ratios = [0.3, 0.4]
for model_name in model_names:
    for test_ratio in test_ratios:
        print(f"training {model_name} {test_ratio} ...")
        current_date = datetime.now().strftime('%Y-%m-%d')
        os.makedirs(f"./output/tl+rotation/{current_date}/{model_name}", exist_ok=True)
        show_ = False
        accuracy_list, precision_list, recall_list, f1_list, tps, tns, fps, fns = train_net(model_name, test_ratio)
        plot_metric(accuracy_list, 'Accuracy', model=model_name, save=True, show=show_, output=f"output/tl+rotation+more/{test_ratio}/{current_date}", num_epochs=num_epochs, test_ratio=test_ratio)
        plot_metric(precision_list, 'Precision', model=model_name, save=True, show=show_,output=f"output/tl+rotation+more/{test_ratio}/{current_date}", num_epochs=num_epochs, test_ratio=test_ratio)
        plot_metric(recall_list, 'Recall', model=model_name, save=True, show=show_,output=f"output/tl+rotation+more/{test_ratio}/{current_date}", num_epochs=num_epochs, test_ratio=test_ratio)
        plot_metric(f1_list, 'F1 Score', model=model_name, save=True, show=show_,output=f"output/tl+rotation+more/{test_ratio}/{current_date}", num_epochs=num_epochs, test_ratio=test_ratio)
        # for time in [1, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 289]:
        # for time in [1]:
            # plot_confusion_matrix(tps, tns, fps, fns, time, save=True, show=show_,output=f"./output/tl+rotation+more/{test_ratio}/{current_date}/{model_name}")
        print(f"{model_name} {test_ratio} {accuracy_list}")
        print(f"{model_name} {test_ratio} average accuracy:", sum(accuracy_list) / len(accuracy_list))

training densenet 0.3 ...
timestamp:  1
Epoch 0/50
train Loss: 1.5929 Acc: 0.2731
val Loss: 1.5051 Acc: 0.2766
----------
Epoch 5/50
train Loss: 0.5205 Acc: 0.8611
val Loss: 0.5487 Acc: 0.8511
----------
Epoch 10/50
train Loss: 0.3693 Acc: 0.8843
val Loss: 0.4403 Acc: 0.7872
----------
Epoch 15/50
train Loss: 0.3576 Acc: 0.9213
val Loss: 0.4306 Acc: 0.8511
----------
Epoch 20/50
train Loss: 0.2570 Acc: 0.9398
val Loss: 0.3574 Acc: 0.8936
----------
Epoch 25/50
train Loss: 0.2143 Acc: 0.9583
val Loss: 0.3447 Acc: 0.9149
----------
Epoch 30/50
train Loss: 0.1727 Acc: 0.9722
val Loss: 0.3764 Acc: 0.8511
----------
Epoch 35/50
train Loss: 0.1941 Acc: 0.9583
val Loss: 0.4173 Acc: 0.8511
----------
Epoch 40/50
train Loss: 0.1675 Acc: 0.9676
val Loss: 0.4088 Acc: 0.8511
----------
Epoch 45/50
train Loss: 0.1550 Acc: 0.9676
val Loss: 0.3659 Acc: 0.9149
----------
Epoch 50/50
train Loss: 0.1521 Acc: 0.9537
val Loss: 0.4218 Acc: 0.8511
----------
Training complete in 2m 15s
Best val Acc: 0.91489

ValueError: too many values to unpack (expected 8)